In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer, AutoModelForSequenceClassification

import pandas as pd

E0000 00:00:1726112669.936638      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0912 03:44:29.944933860      77 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0912 03:44:29.944959515      77 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0912 03:44:29.944963145      77 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0912 03:44:29.944965578      77 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0912 03:44:29.944967925      77 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
test=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")

In [3]:
premise=test['premise'].tolist()

In [4]:
hypothesis=test['hypothesis'].tolist()

In [5]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# Load the tokenizer and model

model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Define the input

# Tokenize input
inputs = tokenizer(premise, hypothesis, truncation=True, padding=True, return_tensors="tf")

# Run inference
outputs = model(inputs)
logits = outputs.logits

# Process results
predictions = tf.nn.softmax(logits, axis=-1)
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(predictions[0].numpy(), label_names)}

print(prediction)


I0000 00:00:1726112705.763979      77 service.cc:145] XLA service 0x5c2a7f58e400 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726112705.764037      77 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1726112705.764042      77 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1726112705.764044      77 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1726112705.764049      77 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1726112705.764052      77 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1726112705.764054      77 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1726112705.764061      77 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1726112705.764064      77 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForSequenceClassification: ['deberta.embeddings.position_ids']
- This IS expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).


All the weights of TFDebertaV2ForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2ForSequenceClassification for predictions without further training.


{'entailment': 0.0, 'neutral': 0.0, 'contradiction': 100.0}


In [6]:
predictions

<tf.Tensor: shape=(5195, 3), dtype=float32, numpy=
array([[2.2774829e-04, 2.2177960e-04, 9.9955040e-01],
       [4.3506917e-04, 9.3541110e-01, 6.4153805e-02],
       [9.9886215e-01, 1.0460539e-03, 9.1794573e-05],
       ...,
       [9.9684924e-01, 1.8347263e-03, 1.3161352e-03],
       [9.9487829e-01, 4.5765685e-03, 5.4511736e-04],
       [1.2930250e-04, 3.0567235e-04, 9.9956506e-01]], dtype=float32)>

In [7]:
import numpy as np
submission = test.id.copy().to_frame()
submission["prediction"] = np.argmax(predictions, axis=1)

submission

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1
...,...,...
5190,5f90dd59b0,0
5191,f357a04e86,2
5192,1f0ea92118,0
5193,0407b48afb,0


In [8]:
submission.to_csv("submission.csv", index=False)